# Reconhecimento de Movimentos Utilizando uma Rede *Long Short-Term Memory* (LSTM)

## *Fine Tuning*

## Carregamento e Tratamento dos *Datasets*

In [1]:
import pandas as pd
from keras.utils import *

# Carregar os datasets
X_treino = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/train/X_train.txt", sep="\s+", header=None)
y_treino = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/train/y_train.txt", sep="\s+", header=None)

X_teste = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/test/X_test.txt", sep="\s+", header=None)
y_teste = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/test/y_test.txt", sep="\s+", header=None)

# Preparar os dados para entrada na rede LSTM
X_treino = X_treino.values.reshape((X_treino.shape[0], 1, X_treino.shape[1]))
X_teste = X_teste.values.reshape((X_teste.shape[0], 1, X_teste.shape[1]))

y_treino = to_categorical(y_treino)
y_teste = to_categorical(y_teste)

X_treino.shape[1], X_treino.shape[2]

(1, 561)

## Criação do Modelo da Rede Neuronal

In [2]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *

def build_model(hp):
    # Definir a arquitetura da rede LSTM
    modelo = Sequential()
    modelo.add(LSTM(hp.Int("units_lstm", min_value=1, max_value=600), input_shape=(X_treino.shape[1], X_treino.shape[2])))
    modelo.add(Dense(hp.Int("neurons_dense", min_value=1, max_value=600), activation=hp.Choice("activation_dense", ["relu", "sigmoid"])))
    modelo.add(Dense(7, activation="softmax"))
    # Compilar o modelo
    modelo.compile(loss=hp.Choice("loss", ['binary_crossentropy', 'sparse_categorical_crossentropy']), optimizer=Adam(learning_rate=hp.Choice("learning_rate", [0.0001, 0.001, 0.01])), metrics=['accuracy'])
    return modelo

## Fine Tuning

In [6]:
from keras_tuner import *

tuner = BayesianOptimization(hypermodel=build_model, objective='accuracy')
tuner.search(X_treino, y_treino, validation_data=(X_teste, y_teste), epochs=20, batch_size=10)

Trial 10 Complete [00h 04m 01s]
accuracy: 0.9861262440681458

Best accuracy So Far: 0.9863982796669006
Total elapsed time: 00h 18m 46s


In [7]:
hyperparameters = tuner.get_best_hyperparameters()[0]

print(f"Unidades LSTM: {hyperparameters.get('units_lstm')}")
print(f"Neurónios Dense: {hyperparameters.get('neurons_dense')}")
print(f"Função de Ativação Dense: {hyperparameters.get('activation_dense')}")
print(f"Função de Perda: {hyperparameters.get('loss')}")
print(f"Learning Rate: {hyperparameters.get('learning_rate')}")

Unidades LSTM: 117
Neurónios Dense: 269
Função de Ativação Dense: relu
Função de Perda: binary_crossentropy
Learning Rate: 0.0001


In [8]:
best_model = tuner.get_best_models()[0]
best_model.summary()

C:\Users\daniel.a.goncalves\Desktop\tedi\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 16 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 117)                 │         317,772 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 269)                 │          31,742 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 7)                   │           1,890 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 351,404 (1.34 MB)

 Trainable params: 351,404 (1.34 MB)

 Non-trainable params: 0 (0.00 B)